In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
url = 'https://mydramalist.com/shows/top?page=1'


In [3]:
def initialize_browser():
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)
    print('Starting Driver')
    title_element = driver.find_element(by=By.CSS_SELECTOR, value='h6.title a')
    title_element.click()

initialize_browser()

'''
this thing from gpt:
    # Wait for the title element to be clickable
    title_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "h6.title a"))
    )

do i ever need to do driver.quit?
'''    

Starting Driver


'\nthis thing from gpt:\n    # Wait for the title element to be clickable\n    title_element = WebDriverWait(driver, 10).until(\n        EC.element_to_be_clickable((By.CSS_SELECTOR, "h6.title a"))\n    )\n\ndo i ever need to do driver.quit?\n'

In [11]:
def initialize_browser():
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)
    print('Starting Driver')
    title_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'h6.title a'))
        )
    title_element.click()
        
    print('moving on to next page')
    content_rating_element = WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, '.show-status + p'))
        )
    content_rating = content_rating_element.text.strip()
    print('Content Rating:', content_rating)
    
initialize_browser()

Starting Driver
moving on to next page


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001052f5d28 chromedriver + 4795688
1   chromedriver                        0x00000001052ed2b3 chromedriver + 4760243
2   chromedriver                        0x0000000104ec688d chromedriver + 407693
3   chromedriver                        0x0000000104f153d9 chromedriver + 730073
4   chromedriver                        0x0000000104f15591 chromedriver + 730513
5   chromedriver                        0x0000000104f5b7e4 chromedriver + 1017828
6   chromedriver                        0x0000000104f3cccd chromedriver + 892109
7   chromedriver                        0x0000000104f58b2b chromedriver + 1006379
8   chromedriver                        0x0000000104f3ca73 chromedriver + 891507
9   chromedriver                        0x0000000104f07143 chromedriver + 672067
10  chromedriver                        0x0000000104f0831e chromedriver + 676638
11  chromedriver                        0x00000001052b6795 chrome

In [5]:
# new attempt
import time
genres = []
def initialize_browser():
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)
    print('Starting Driver')
    title_element = driver.find_element(by=By.CSS_SELECTOR, value='h6.title a')
    time.sleep(.5)
    title_element.click()
    time.sleep(1)
    print('moving on to next page')
    genres_section = driver.find_element(by=By.CLASS_NAME, value='show-genres')
    genres = genres_section[0].find_elements(By.CLASS_NAME, 'badge')
    for genre in genres:
        genres.append(genre)
    
initialize_browser()

Starting Driver
moving on to next page


TypeError: 'WebElement' object is not subscriptable

In [ ]:
By.CLASS_NAME, 'show-genres'